In [1]:
from datetime import datetime
import pandas as pd
import glob

In [2]:
experiments = glob.glob("C:/Users/hzwaan/Documents/serverlessgraphs/timestamps/benchmark_runs/*")

FORM = "%H:%M:%S.%f"
PARAMETERS = ["lpid", "lpps", "rpid", "rpps", "spps"]
FULL_PARAMETR_NAME = { "lpid": "livenessProbeInitialDelay", 
                       "lpps": "livenessProbePeriodSeconds", 
                       "rpid": "readinessProbeInitialDelay", 
                       "rpps": "readinessProbePeriodSeconds", 
                       "spps": "startupProbe.periodSeconds"}

In [6]:
# For all experiments
for experiment in experiments:
    try:
        # Get all csv's and txt's
        total_time = glob.glob(f"{experiment}\*.csv")
        execution_time = glob.glob(f"{experiment}\*.txt")

        flag = False
        for t in total_time:
            if "clean_df.csv" in t:
                flag = True
                break
        if flag:
            continue

        if len(total_time) == 0:
            continue

        experiment_name = experiment.split('\\')[1]
        clean_df = None    

        # Do this for all 10 runs
        for i, run in enumerate(total_time):
            # Get csv path and corrosponding txt path
            csv_path = run.split("\\", 1)[1]
            indentifier = csv_path.split("-",2)[2].split(".")[0]
            txt_path = None
            
            for tp_time in execution_time:
                if indentifier in tp_time:
                    txt_path = tp_time.split("\\", 1)[1]
                    break

            # Clean dataframe txt (azure time is 2 hours behind on local time)
            execute = pd.read_csv(f"benchmark_runs/{txt_path}", sep=" ", header=None)
            execute.columns = ["Date", "Time"]
            # Start = the tenant program in the container start working
            # Dataframe = the tenant dataframe is created and can be executed
            # Finished = The tenant program has been executed and results are saved
            execute.insert(0, "Action", ["start", "dataframe", "finished"])
            execute["Time"] = execute["Time"] + pd.Timedelta(hours=2)
            execute["Time"] = execute["Time"].astype(str).replace("0 days ", "", regex=True)
            execute = execute.drop('Date', axis=1)

            # Clean dataframe
            run = pd.read_csv(f"benchmark_runs/{csv_path}")
            run = run.drop('Date', axis=1)

            # Calcultate the duration
            creation = datetime.strptime(run.iloc[1]["Time"], FORM) - datetime.strptime(run.iloc[0]["Time"], FORM)
            tenant_program_start = datetime.strptime(execute.iloc[0]["Time"], FORM) - datetime.strptime(run.iloc[2]["Time"], FORM)
            tenant_program_dataframe_creation = datetime.strptime(execute.iloc[1]["Time"], FORM) - datetime.strptime(execute.iloc[0]["Time"], FORM)
            tenant_program_execution = datetime.strptime(execute.iloc[2]["Time"], FORM) - datetime.strptime(execute.iloc[1]["Time"], FORM)
            execution = datetime.strptime(run.iloc[3]["Time"], FORM) - datetime.strptime(run.iloc[2]["Time"], FORM)
            deletion = datetime.strptime(run.iloc[5]["Time"], FORM) - datetime.strptime(run.iloc[4]["Time"], FORM)

            # Create readable dataframe
            result_total = pd.DataFrame([creation, execution, deletion, tenant_program_start, tenant_program_dataframe_creation, tenant_program_execution])
            result_total.columns = ["Time"]
            result_total.index = ["creation", "execution", "deletion", "tpstart", "tpdataframe", "tpfinished"]
            result_total["Time"] = result_total["Time"].astype(str).replace("0 days ", "", regex=True)
            result_total = result_total.transpose()
            result_total["identifier"] = indentifier
            result_total.index = [f"run {i+1}"]

            if i == 0:
                clean_df = result_total
            else:
                clean_df = clean_df._append(result_total, ignore_index=True)

        clean_df.to_csv(f"benchmark_runs/{experiment_name}/clean_df.csv")
    except Exception as e:
        print(e)